In [1]:
#library

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import os
import csv
import time
import random
import pandas as pd

# Retrieve Data

In [3]:
def filechecker(name):
    data=pd.read_csv(name)
    print('top 5 row of',len(data), 'data\n',data.head())

In [4]:
#get data

In [5]:
url='https://bulbapedia.bulbagarden.net/wiki/List_of_Pokémon_by_National_Pokédex_number'
r=requests.get(url)
r.encoding = 'utf-8'

In [6]:
#compile data
soup=BeautifulSoup(r.text,"html5lib")

In [7]:
#create folder
folder = os.path.exists('Pokemon_data')
if not folder:
#if not exist creat one
    os.makedirs('Pokemon_data')

In [8]:
#create file
f = open("Pokemon_data/" + 'pokemon_data.csv', "a+"
                     #open for writing, appending to the end of the file if it exists
                     #open for updating (reading and writing)
                     #https://docs.python.org/3/library/functions.html
                     , encoding='utf-8')

In [9]:
#open file and create column names
with open('Pokemon_data/pokemon_data.csv', 'a', newline='') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow(['Kdex','Ndex','Name','Type1','Type2','Link',
               'HP','ATK','DEF','SP_ATK','SP_DEF','SPD',
               'Damaged Normally by','Weak to','Immue to','Resisant to'])

In [10]:
#arrange data
counter=0 #counter
for i in soup.find('div',{'id':'mw-content-text'}).find_all('tr',{"style":"background:#FFF"}):
    counter=counter+1 #counter
    pokemon_data=[]    
    #generate_number
    generate_number=i.find_all('td')[0].text.replace('\n',"")
    #print('Kdex: ',generate_number)
    pokemon_data.append(generate_number)
    
    
    #national_number
    national_number=i.find_all('td')[1].text.replace('\n',"")
    #print('Ndex: ',national_number)
    pokemon_data.append(national_number)
    
    
    #name
    name=i.find('a')['title']
    #print('Name: ',name)
    pokemon_data.append(name)
    print('Loading...',name,"'s Data")
    
    
    #type1
    type1=i.find_all('span')[0].text
    #print('Type1: ',type1)
    pokemon_data.append(type1)
    
    
    #type2 if exist
    if len(i.find_all('span'))==2:
        type2=i.find_all('span')[1].text
        #print('Type2: ',type2)
        pokemon_data.append(type2)
    else:
        type2='Null'
        #print('Type2: ',type2)
        pokemon_data.append(type2)
        
        
    #link to specific attr
    link='https://bulbapedia.bulbagarden.net/'+i.find_all('a',
                                                          href=re.compile("^(/wiki/)((?!;)\S)*$"))[1]['href']
    #print('Link: ',link)
    pokemon_data.append(link)    
    
    
    
    #set timer 
    sleep=random.randint(0,1)
    time.sleep(sleep)
    #print('sleep time: ',sleep)
    
    
    #get specific attr for a pokemon
    url_pkm=link
    r_pkm=requests.get(url_pkm)
    r_pkm.encoding = 'utf-8'
    soup_pkm=BeautifulSoup(r_pkm.text,"html5lib")
    try:
        #hp
        hp=soup_pkm.find('tr',{'style':"background: #FF5959; text-align:center"}).find('div',{'style':'float:right'}).text
        #print("HP: ",hp) 
        pokemon_data.append(hp)
        #atk
        atk=soup_pkm.find('tr',{'style':"background: #F5AC78; text-align:center"}).find('div',{'style':'float:right'}).text
        #print("ATK: ",atk) 
        pokemon_data.append(atk)
        #def
        defense=soup_pkm.find('tr',{'style':"background: #FAE078; text-align:center"}).find('div',{'style':'float:right'}).text
        #print("DEF: ",defense)
        pokemon_data.append(defense)
        #sp_atk
        sp_atk=soup_pkm.find('tr',{'style':"background: #9DB7F5; text-align:center"}).find('div',{'style':'float:right'}).text
        #print("SP_ATK: ",sp_atk)  
        pokemon_data.append(sp_atk)
        #sp_def
        sp_defense=soup_pkm.find('tr',{'style':"background: #A7DB8D; text-align:center"}).find('div',{'style':'float:right'}).text
        #print("SP_DEF: ",sp_defense)
        pokemon_data.append(sp_defense)
        #spd
        spd=soup_pkm.find('tr',{'style':"background: #FA92B2; text-align:center"}).find('div',{'style':'float:right'}).text
        #print("SPD: ",spd)
        pokemon_data.append(spd)

        
        #type_effectiveness
        type_effectiveness=soup_pkm.find('table',{'class':'roundy','width':'100%','align':'center'}) 
        #get four dimentions 
        normal=type_effectiveness.find_all('table',{'class':'roundy','width':'100%','align':'center'})[1]
        weak=type_effectiveness.find_all('table',{'class':'roundy','width':'100%','align':'center'})[2]
        immue=type_effectiveness.find_all('table',{'class':'roundy','width':'100%','align':'center'})[3]
        resisant=type_effectiveness.find_all('table',{'class':'roundy','width':'100%','align':'center'})[4]
        #create list for each dimention
        normal_lst=[]
        weak_lst=[]
        immue_lst=[]
        resisant_lst=[]
    
    
        #get data into each list and write
        #Damaged Normally by
        for j in normal.find_all('span',{'style':'display:inline-block;'}):
            normal_lst.append(j.text.replace('\n','').strip().replace(' ','_'))
        #print('Damaged Normally by :',normal_lst)
        pokemon_data.append(str(normal_lst))
    
    
        #Weak to
        for j in weak.find_all('span',{'style':'display:inline-block;'}):
            weak_lst.append(j.text.replace('\n','').strip().replace(' ','_'))
        #print('Weak to :',weak_lst)
        pokemon_data.append(str(weak_lst))
    
    
        #Immue to
        for j in immue.find_all('span',{'style':'display:inline-block;'}):
            immue_lst.append(j.text.replace('\n','').strip().replace(' ','_'))
        #print('Immue to :',immue_lst)
        pokemon_data.append(str(immue_lst))
    
    
        #Resisant to
        for j in resisant.find_all('span',{'style':'display:inline-block;'}):
            resisant_lst.append(j.text.replace('\n','').strip().replace(' ','_'))
        #print('Resisant to :',resisant_lst)
        pokemon_data.append(str(resisant_lst))
    
    
    #writting data    
        with open('Pokemon_data/pokemon_data.csv', 'a', newline='') as file:
            writer = csv.writer(file, delimiter=',')
            writer.writerow(pokemon_data)
    except:
        continue

print('Done!!!')
    
        
        


Loading... Bulbasaur 's Data
Loading... Ivysaur 's Data
Loading... Venusaur 's Data
Loading... Charmander 's Data
Loading... Charmeleon 's Data
Loading... Charizard 's Data
Loading... Squirtle 's Data
Loading... Wartortle 's Data
Loading... Blastoise 's Data
Loading... Caterpie 's Data
Loading... Metapod 's Data
Loading... Butterfree 's Data
Loading... Weedle 's Data
Loading... Kakuna 's Data
Loading... Beedrill 's Data
Loading... Pidgey 's Data
Loading... Pidgeotto 's Data
Loading... Pidgeot 's Data
Loading... Rattata 's Data
Loading... Rattata 's Data
Loading... Raticate 's Data
Loading... Raticate 's Data
Loading... Spearow 's Data
Loading... Fearow 's Data
Loading... Ekans 's Data
Loading... Arbok 's Data
Loading... Pikachu 's Data
Loading... Raichu 's Data
Loading... Raichu 's Data
Loading... Sandshrew 's Data
Loading... Sandshrew 's Data
Loading... Sandslash 's Data
Loading... Sandslash 's Data
Loading... Nidoran♀ 's Data
Loading... Nidorina 's Data
Loading... Nidoqueen 's Data
L

Loading... Cascoon 's Data
Loading... Dustox 's Data
Loading... Lotad 's Data
Loading... Lombre 's Data
Loading... Ludicolo 's Data
Loading... Seedot 's Data
Loading... Nuzleaf 's Data
Loading... Shiftry 's Data
Loading... Taillow 's Data
Loading... Swellow 's Data
Loading... Wingull 's Data
Loading... Pelipper 's Data
Loading... Ralts 's Data
Loading... Kirlia 's Data
Loading... Gardevoir 's Data
Loading... Surskit 's Data
Loading... Masquerain 's Data
Loading... Shroomish 's Data
Loading... Breloom 's Data
Loading... Slakoth 's Data
Loading... Vigoroth 's Data
Loading... Slaking 's Data
Loading... Nincada 's Data
Loading... Ninjask 's Data
Loading... Shedinja 's Data
Loading... Whismur 's Data
Loading... Loudred 's Data
Loading... Exploud 's Data
Loading... Makuhita 's Data
Loading... Hariyama 's Data
Loading... Azurill 's Data
Loading... Nosepass 's Data
Loading... Skitty 's Data
Loading... Delcatty 's Data
Loading... Sableye 's Data
Loading... Mawile 's Data
Loading... Aron 's Data

Loading... Lilligant 's Data
Loading... Basculin 's Data
Loading... Basculin 's Data
Loading... Sandile 's Data
Loading... Krokorok 's Data
Loading... Krookodile 's Data
Loading... Darumaka 's Data
Loading... Darumaka 's Data
Loading... Darmanitan 's Data
Loading... Darmanitan 's Data
Loading... Darmanitan 's Data
Loading... Darmanitan 's Data
Loading... Maractus 's Data
Loading... Dwebble 's Data
Loading... Crustle 's Data
Loading... Scraggy 's Data
Loading... Scrafty 's Data
Loading... Sigilyph 's Data
Loading... Yamask 's Data
Loading... Yamask 's Data
Loading... Cofagrigus 's Data
Loading... Tirtouga 's Data
Loading... Carracosta 's Data
Loading... Archen 's Data
Loading... Archeops 's Data
Loading... Trubbish 's Data
Loading... Garbodor 's Data
Loading... Zorua 's Data
Loading... Zoroark 's Data
Loading... Minccino 's Data
Loading... Cinccino 's Data
Loading... Gothita 's Data
Loading... Gothorita 's Data
Loading... Gothitelle 's Data
Loading... Solosis 's Data
Loading... Duosion 

Loading... Eldegoss 's Data
Loading... Wooloo 's Data
Loading... Dubwool 's Data
Loading... Chewtle 's Data
Loading... Drednaw 's Data
Loading... Yamper 's Data
Loading... Boltund 's Data
Loading... Rolycoly 's Data
Loading... Carkol 's Data
Loading... Coalossal 's Data
Loading... Applin 's Data
Loading... Flapple 's Data
Loading... Appletun 's Data
Loading... Silicobra 's Data
Loading... Sandaconda 's Data
Loading... Cramorant 's Data
Loading... Arrokuda 's Data
Loading... Barraskewda 's Data
Loading... Toxel 's Data
Loading... Toxtricity 's Data
Loading... Sizzlipede 's Data
Loading... Centiskorch 's Data
Loading... Clobbopus 's Data
Loading... Grapploct 's Data
Loading... Sinistea 's Data
Loading... Polteageist 's Data
Loading... Hatenna 's Data
Loading... Hattrem 's Data
Loading... Hatterene 's Data
Loading... Impidimp 's Data
Loading... Morgrem 's Data
Loading... Grimmsnarl 's Data
Loading... Obstagoon 's Data
Loading... Perrserker 's Data
Loading... Cursola 's Data
Loading... Sir

In [11]:
f.close()

In [13]:
filechecker('Pokemon_data/pokemon_data.csv')


top 5 row of 1261 data
     Kdex   Ndex        Name  Type1   Type2  \
0   #001   #001   Bulbasaur  Grass  Poison   
1   #002   #002     Ivysaur  Grass  Poison   
2   #003   #003    Venusaur  Grass  Poison   
3   #004   #004  Charmander   Fire    Null   
4   #005   #005  Charmeleon   Fire    Null   

                                                Link  HP ATK DEF SP_ATK  \
0  https://bulbapedia.bulbagarden.net//wiki/Bulba...  45  49  49     65   
1  https://bulbapedia.bulbagarden.net//wiki/Ivysa...  60  62  63     80   
2  https://bulbapedia.bulbagarden.net//wiki/Venus...  80  82  83    100   
3  https://bulbapedia.bulbagarden.net//wiki/Charm...  39  52  43     60   
4  https://bulbapedia.bulbagarden.net//wiki/Charm...  58  64  58     80   

  SP_DEF SPD                                Damaged Normally by  \
0     65  45  ['Normal_1×', 'Poison_1×', 'Ground_1×', 'Rock_...   
1     80  60  ['Normal_1×', 'Poison_1×', 'Ground_1×', 'Rock_...   
2    100  80  ['Normal_1×', 'Poison_1×', 'Groun

# Import Data into MongoDB

In [34]:
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
STATIC_DIR=os.path.abspath('/static')
app = Flask(__name__)

mongo = PyMongo(app, uri="mongodb://localhost:27017/Pokemon_DB")

In [35]:
pokemon_data=pd.read_csv("Pokemon_data/pokemon_data.csv")
pokemon_data.head()

,Kdex,Ndex,Name,Type1,Type2,Link,HP,ATK,DEF,SP_ATK,SP_DEF,SPD,Damaged Normally by,Weak to,Immue to,Resisant to
0,#001,#001,Bulbasaur,Grass,Poison,https://bulbapedia.bulbagarden.net//wiki/Bulba...,45,49,49,65,65,45,"['Normal_1×', 'Poison_1×', 'Ground_1×', 'Rock_...","['Flying_2×', 'Fire_2×', 'Psychic_2×', 'Ice_2×']",['None'],"['Fighting_½×', 'Water_½×', 'Grass_¼×', 'Elect..."
1,#002,#002,Ivysaur,Grass,Poison,https://bulbapedia.bulbagarden.net//wiki/Ivysa...,60,62,63,80,80,60,"['Normal_1×', 'Poison_1×', 'Ground_1×', 'Rock_...","['Flying_2×', 'Fire_2×', 'Psychic_2×', 'Ice_2×']",['None'],"['Fighting_½×', 'Water_½×', 'Grass_¼×', 'Elect..."
2,#003,#003,Venusaur,Grass,Poison,https://bulbapedia.bulbagarden.net//wiki/Venus...,80,82,83,100,100,80,"['Normal_1×', 'Poison_1×', 'Ground_1×', 'Rock_...","['Flying_2×', 'Fire_2×', 'Psychic_2×', 'Ice_2×']",['None'],"['Fighting_½×', 'Water_½×', 'Grass_¼×', 'Elect..."
3,#004,#004,Charmander,Fire,Null,https://bulbapedia.bulbagarden.net//wiki/Charm...,39,52,43,60,50,65,"['Normal_1×', 'Fighting_1×', 'Flying_1×', 'Poi...","['Ground_2×', 'Rock_2×', 'Water_2×']",['None'],"['Bug_½×', 'Steel_½×', 'Fire_½×', 'Grass_½×', ..."
4,#005,#005,Charmeleon,Fire,Null,https://bulbapedia.bulbagarden.net//wiki/Charm...,58,64,58,80,65,80,"['Normal_1×', 'Fighting_1×', 'Flying_1×', 'Poi...","['Ground_2×', 'Rock_2×', 'Water_2×']",['None'],"['Bug_½×', 'Steel_½×', 'Fire_½×', 'Grass_½×', ..."


In [37]:
id_count=0
for i in range(len(pokemon_data)):
    id_count=id_count+1
    data_list=[id_count]
    data_list.extend(pokemon_data.iloc[i,:6])
    data_list.append(int(pokemon_data.iloc[i,6]))
    data_list.append(int(pokemon_data.iloc[i,7]))
    data_list.append(int(pokemon_data.iloc[i,8]))
    data_list.append(int(pokemon_data.iloc[i,9]))
    data_list.append(int(pokemon_data.iloc[i,10]))
    data_list.append(int(pokemon_data.iloc[i,11]))
    data_list.extend(pokemon_data.iloc[i,12:])
    
    column_list=['Id']
    column_list.extend(list(pokemon_data.columns))
    pokemon_data_dict={}
    for d,c in zip(data_list,column_list):
        pokemon_data_dict.update({c:d})
    mongo.db.Pokemon_Table.update_one({}, pokemon_data_dict, upsert=True)
    

/Users/mac/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  import sys


# test section

In [11]:

import pandas as pd
pokemon_data=pd.read_csv("Pokemon_data/pokemon_data.csv")
pokemon_data.head()

,Kdex,Ndex,Name,Type1,Type2,Link,HP,ATK,DEF,SP_ATK,SP_DEF,SPD,Damaged Normally by,Weak to,Immue to,Resisant to
0,#001,#001,Bulbasaur,Grass,Poison,https://bulbapedia.bulbagarden.net//wiki/Bulba...,45,49,49,65,65,45,"['Normal_1×', 'Poison_1×', 'Ground_1×', 'Rock_...","['Flying_2×', 'Fire_2×', 'Psychic_2×', 'Ice_2×']",['None'],"['Fighting_½×', 'Water_½×', 'Grass_¼×', 'Elect..."
1,#002,#002,Ivysaur,Grass,Poison,https://bulbapedia.bulbagarden.net//wiki/Ivysa...,60,62,63,80,80,60,"['Normal_1×', 'Poison_1×', 'Ground_1×', 'Rock_...","['Flying_2×', 'Fire_2×', 'Psychic_2×', 'Ice_2×']",['None'],"['Fighting_½×', 'Water_½×', 'Grass_¼×', 'Elect..."
2,#003,#003,Venusaur,Grass,Poison,https://bulbapedia.bulbagarden.net//wiki/Venus...,80,82,83,100,100,80,"['Normal_1×', 'Poison_1×', 'Ground_1×', 'Rock_...","['Flying_2×', 'Fire_2×', 'Psychic_2×', 'Ice_2×']",['None'],"['Fighting_½×', 'Water_½×', 'Grass_¼×', 'Elect..."
3,#004,#004,Charmander,Fire,Null,https://bulbapedia.bulbagarden.net//wiki/Charm...,39,52,43,60,50,65,"['Normal_1×', 'Fighting_1×', 'Flying_1×', 'Poi...","['Ground_2×', 'Rock_2×', 'Water_2×']",['None'],"['Bug_½×', 'Steel_½×', 'Fire_½×', 'Grass_½×', ..."
4,#005,#005,Charmeleon,Fire,Null,https://bulbapedia.bulbagarden.net//wiki/Charm...,58,64,58,80,65,80,"['Normal_1×', 'Fighting_1×', 'Flying_1×', 'Poi...","['Ground_2×', 'Rock_2×', 'Water_2×']",['None'],"['Bug_½×', 'Steel_½×', 'Fire_½×', 'Grass_½×', ..."


In [17]:
pokemon_data.columns

Index(['Kdex', 'Ndex', 'Name', 'Type1', 'Type2', 'Link', 'HP', 'ATK', 'DEF',
       'SP_ATK', 'SP_DEF', 'SPD', 'Damaged Normally by', 'Weak to', 'Immue to',
       'Resisant to'],
      dtype='object')

In [15]:
type1=list(pokemon_data['Type1'].unique())
type2=list(pokemon_data['Type2'].unique())
type_list=[]
type_list.extend(type1)
for t in type2:
    if t not in type1:
        type_list.append(t)
type_list


['Grass',
 'Fire',
 'Water',
 'Bug',
 'Normal',
 'Dark',
 'Poison',
 'Electric',
 'Ground',
 'Ice',
 'Fairy',
 'Steel',
 'Fighting',
 'Psychic',
 'Rock',
 'Ghost',
 'Dragon',
 'Flying',
 'Null']

In [18]:
pokemon_data

,Kdex,Ndex,Name,Type1,Type2,Link,HP,ATK,DEF,SP_ATK,SP_DEF,SPD,Damaged Normally by,Weak to,Immue to,Resisant to
0,#001,#001,Bulbasaur,Grass,Poison,https://bulbapedia.bulbagarden.net//wiki/Bulba...,45,49,49,65,65,45,"['Normal_1×', 'Poison_1×', 'Ground_1×', 'Rock_...","['Flying_2×', 'Fire_2×', 'Psychic_2×', 'Ice_2×']",['None'],"['Fighting_½×', 'Water_½×', 'Grass_¼×', 'Elect..."
1,#002,#002,Ivysaur,Grass,Poison,https://bulbapedia.bulbagarden.net//wiki/Ivysa...,60,62,63,80,80,60,"['Normal_1×', 'Poison_1×', 'Ground_1×', 'Rock_...","['Flying_2×', 'Fire_2×', 'Psychic_2×', 'Ice_2×']",['None'],"['Fighting_½×', 'Water_½×', 'Grass_¼×', 'Elect..."
2,#003,#003,Venusaur,Grass,Poison,https://bulbapedia.bulbagarden.net//wiki/Venus...,80,82,83,100,100,80,"['Normal_1×', 'Poison_1×', 'Ground_1×', 'Rock_...","['Flying_2×', 'Fire_2×', 'Psychic_2×', 'Ice_2×']",['None'],"['Fighting_½×', 'Water_½×', 'Grass_¼×', 'Elect..."
3,#004,#004,Charmander,Fire,Null,https://bulbapedia.bulbagarden.net//wiki/Charm...,39,52,43,60,50,65,"['Normal_1×', 'Fighting_1×', 'Flying_1×', 'Poi...","['Ground_2×', 'Rock_2×', 'Water_2×']",['None'],"['Bug_½×', 'Steel_½×', 'Fire_½×', 'Grass_½×', ..."
4,#005,#005,Charmeleon,Fire,Null,https://bulbapedia.bulbagarden.net//wiki/Charm...,58,64,58,80,65,80,"['Normal_1×', 'Fighting_1×', 'Flying_1×', 'Poi...","['Ground_2×', 'Rock_2×', 'Water_2×']",['None'],"['Bug_½×', 'Steel_½×', 'Fire_½×', 'Grass_½×', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965,#---,#896,Glastrier,Ice,Null,https://bulbapedia.bulbagarden.net//wiki/Glast...,100,145,130,65,110,30,"['Normal_1×', 'Flying_1×', 'Poison_1×', 'Groun...","['Fighting_2×', 'Rock_2×', 'Steel_2×', 'Fire_2×']",['None'],['Ice_½×']
966,#---,#897,Spectrier,Ghost,Null,https://bulbapedia.bulbagarden.net//wiki/Spect...,100,65,60,145,80,130,"['Flying_1×', 'Ground_1×', 'Rock_1×', 'Steel_1...","['Ghost_2×', 'Dark_2×']","['Normal_0×', 'Fighting_0×']","['Poison_½×', 'Bug_½×']"
967,#---,#898,Calyrex,Psychic,Grass,https://bulbapedia.bulbagarden.net//wiki/Calyr...,100,80,80,80,80,80,"['Normal_1×', 'Rock_1×', 'Steel_1×', 'Dragon_1...","['Flying_2×', 'Poison_2×', 'Bug_4×', 'Ghost_2×...",['None'],"['Fighting_½×', 'Ground_½×', 'Water_½×', 'Gras..."
968,#---,#898,Calyrex,Psychic,Ice,https://bulbapedia.bulbagarden.net//wiki/Calyr...,100,80,80,80,80,80,"['Normal_1×', 'Rock_1×', 'Steel_1×', 'Dragon_1...","['Flying_2×', 'Poison_2×', 'Bug_4×', 'Ghost_2×...",['None'],"['Fighting_½×', 'Ground_½×', 'Water_½×', 'Gras..."
